# 导入工具

In [326]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import sys

from gensim import models, corpora
from sklearn.feature_extraction.text import TfidfVectorizer

import jieba
# 基于textrank和tfidf的关键词提取
from jieba.analyse import textrank
from jieba.analyse import extract_tags

from scipy.spatial.distance import cosine

from functools import reduce

import logging
logging.basicConfig(format="%(asctime)s: %(levelname)s: %(message)s", level=logging.INFO)

from IPython.display import display, Markdown

In [2]:
data_file = r"D:\Github\NLP\Artificial_Intelligence_for_NLP\Projects\2_autoSummarization\data\cleaned_data.csv"

In [3]:
data = pd.read_csv(data_file, index_col=0, nrows=10000)

In [4]:
data.tail(3)

,content,title,summary
9997,新华社 照片 ， 武汉 ， 2017 年 4 月 2 日 （ 体育 ） （ 11 ） 足球 ...,（体育）（11）足球——中甲：武汉卓尔胜北京北控,新华社照片，武汉，2017年4月2日 （体育）（11）足球——中甲：武汉卓尔胜北京北控 4月...
9998,新华社 照片 ， 武汉 ， 2017 年 4 月 2 日 （ 体育 ） （ 4 ） 足球 —...,（体育）（4）足球——中甲：武汉卓尔胜北京北控,新华社照片，武汉，2017年4月2日 （体育）（4）足球——中甲：武汉卓尔胜北京北控 4月2...
9999,新华社 照片 ， 武汉 ， 2017 年 4 月 2 日 （ 体育 ） （ 6 ） 足球 —...,（体育）（6）足球——中甲：武汉卓尔胜北京北控,新华社照片，武汉，2017年4月2日 （体育）（6）足球——中甲：武汉卓尔胜北京北控 4月2...


## Demo

In [161]:
new_vectorizer = TfidfVectorizer()
contents = [
    '我 今天 很 难过',
    '你 昨天 挺 开心',
    '他 难过 不 开心。'
]
vecs = new_vectorizer.fit_transform(contents).toarray()

In [162]:
vecs

array([[0.79596054, 0.        , 0.        , 0.60534851],
       [0.        , 0.60534851, 0.79596054, 0.        ],
       [0.        , 0.70710678, 0.        , 0.70710678]])

In [163]:
new_vectorizer.vocabulary_

{'今天': 0, '难过': 3, '昨天': 2, '开心': 1}

---

In [36]:
vectorizer = TfidfVectorizer(max_features=10000)
vec = vectorizer.fit_transform(data.content)
word2index = vectorizer.vocabulary_
index2word = {k:v for v, k in word2index.items()}
vec.shape, len(word2index)

((10000, 10000), 10000)

In [37]:
data.content[2]

'此前 的 一加 3T 搭载 的 是 3400mAh 电池 ， DashCharge 快充 规格 为 5V / 4A 。 至于 电池 缩水 ， 可能 与 刘作 虎 所说 ， 一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关 。 按照 目前 掌握 的 资料 ， 一加 手机 5 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 、 6G / 8GB RAM ， 64GB / 128GB ROM ， 双 1600 万 摄像头 ， 备货 量 “ 惊喜 ” 。 根据 京东 泄露 的 信息 ， 一加 5 起 售价 是 xx99 元 ， 应该 是 在 2799 / 2899 / 2999 中 的 某个 。 '

In [90]:
def vec2doc(vector):
    return ' '.join([index2word[x] for x in np.where(vector.toarray())[1]])

In [80]:
# tfidf提取后的文章，可替代方案：textrank的关键字提取或者jieba中的tfidf关键词提取
vec2doc(vec[1])

'10 30 cpu windows 一些 一款 一种 三星 不会 也许 产品 今年 他们 以便 传统 作为 兼顾 华为 只是 可以 唯一 因为 处理器 完美 小米 已经 平台 开始 强调 当然 形态 微软 性能 找到 报道 拿到 按计划 方案 显示 更好 未来 架构 核心 澎湃 理解 电脑 相反 空间 考虑 联想 联手 至少 节省 见到 认证 资料 进入 适合 通过 集成 预计 首发 骁龙 高通 麒麟'

In [34]:
# textrank关键字提取，可不提前分词，jieba自动分词后再提取，词间距离基于共现矩阵
textrank(data.content[2], withWeight=True)

[('缩水', 1.0),
 ('泄露', 1.0),
 ('设定', 0.8859957719401048),
 ('旗舰', 0.8799957379814727),
 ('资料', 0.6699879211644604),
 ('拥有', 0.6699879211644604),
 ('摄像头', 0.6699879211644604),
 ('掌握', 0.665196407666107),
 ('手机', 0.665196407666107),
 ('备货', 0.665196407666107),
 ('电池', 0.5016810613367134),
 ('可能', 0.5016810613367134),
 ('京东', 0.5016810613367134),
 ('信息', 0.5016810613367134),
 ('有关', 0.4542835271882948),
 ('大屏', 0.45075410554571776)]

In [33]:
# tfidf关键字提取
extract_tags(data.content[2], withWeight=True)

[('一加', 0.8104927120610169),
 ('电池', 0.2548766655786441),
 ('手机', 0.20921429925830506),
 ('3T', 0.20262317801525423),
 ('3400mAh', 0.20262317801525423),
 ('DashCharge', 0.20262317801525423),
 ('快充', 0.20262317801525423),
 ('5V', 0.20262317801525423),
 ('4A', 0.20262317801525423),
 ('刘作', 0.20262317801525423),
 ('大屏', 0.20262317801525423),
 ('5.5', 0.20262317801525423),
 ('1080P', 0.20262317801525423),
 ('AMOLED', 0.20262317801525423),
 ('6G', 0.20262317801525423),
 ('8GB', 0.20262317801525423),
 ('RAM', 0.20262317801525423),
 ('64GB', 0.20262317801525423),
 ('128GB', 0.20262317801525423),
 ('ROM', 0.20262317801525423)]

In [113]:
random_index = np.random.randint(0,10000)
sample = data.content[random_index]
sample, random_index

('参考消息 网 6 月 21 日 报道 韩媒称 ， 韩国 总统 文在寅 20 日 在 接受 《 华盛顿邮报 》 专访 时 就 如何 看待 在 朝鲜半岛 部署 “ 萨德 ” 反导 系统 的 提问 表示 ， 环境影响 评估 并 不 意味着 推迟 或 取消 部署 “ 萨德 ” 。 据 韩联社 6 月 21 日 报道 ， 文在寅 表示 ， 韩美 已 部署 “ 萨德 ” 雷达 系统 和 两辆 发射 车 ， 但 还要 走 完 包括 环评 在内 的 法律 程序 。 他 说 ， 在 朝鲜半岛 部署 “ 萨德 ” 是 由 前届 政府 作出 的 决定 ， 他 已经 明确 表明 不会 轻视 这一 决定 。 韩国 总统 文在寅 6 月 15 日 表示 ， 如果 朝鲜 不再 进行 核导 开发 ， 韩方 愿意 与 朝鲜 无条件 开展 对话 。 新华社 发 点击 图片 进入 下 一页 ',
 6604)

In [105]:
# 找出和某一篇文章最近的另一篇文章
def get_most_similarity(doc):
    vector = vectorizer.transform([doc])
    result = [(index, cosine(vector.toarray(), candi.toarray())) 
              for index, candi in enumerate(vec)]
    sorted_result = sorted(result, key=lambda x:x[1])[1][0]
    print(data.content[sorted_result])
    return sorted_result

In [114]:
%time get_most_similarity(sample)

 近期 ， 来自 韩美 有关 “ 萨德 ” 问题 的 消息 很多 ， 不断 冲击 着 中国 的 舆论 和 中国 人民 的 心理 底线 。 美国 总统 特朗普 6 月 8 日 与 国务卿 蒂勒 森 、 国防部长 马蒂斯 等 核心 高官 及 参谋 一起 就 文在寅 政府 决定 先 环评 再 部署 “ 萨德 ” 进行 了 讨论 ， 对此 ， 青瓦台 高官 次日 就 “ 萨德 ” 争议 表示 ， 韩国政府 无意 从根本上 改变 韩美 同盟 之间 约定 的 事项 。 韩国 新任 防长 提名 人 宋永武 所谓 的 买下 “ 萨德 ” 的 议案 重新 抬头 ， 等等 。 围绕 “ 萨德 ” 问题 的 争议 似乎 变得 越来越 复杂 ， 形势 越来越 严峻 。 文在寅 政府 在 “ 萨德 ” 问题 上 的 三种 选择 文在寅 政府 今后 如何 处理 “ 萨德 ” 问题 ， 将 被 国际 社会 所 关注 ， 对 他 来说 或许 是 一个 痛苦 的 抉择 。 不管 问题 有 多么 复杂 ， 一般来讲 ， 不外乎 有 以下 三种 可能性 ： 一是 完全 按照 中方 提出 的 要求 ， 撤除 “ 萨德 ” 系统 。 客观 上 ， 在 朝鲜半岛 乃至 东北亚 的 紧张局势 下 ， 韩国 存在 一定 的 支持 部署 “ 萨德 ” 的 民意 ， 特别 是 来自 现有 韩美 同盟 机制 的 压力 。 朝鲜战争 结束 以来 ， 韩国 军队 的 战时 指挥权 至今 尚 在 美军 手中 。 韩国 前 总统 卢武铉 当政 期间 ， 已经 与 美方 达成协议 将 战时 指挥权 适时 交还 韩方 ， 后来 ， 保守 的 李明博 政府 和 朴槿惠 政府 将 此 期限 相继 延期 。 所以 ， 在 朝 韩 互信 远 未 建立 、 同盟 机制 压力 与日俱增 的 背景 下 ， 文在寅 似乎 很难 做到 撤除 “ 萨德 ” 。 冒 着 与 美方 翻脸 的 风险 ， 文在寅 恐怕 不敢 。 二是 完全 满足 美方 的 要求 ， 继续 完善 “ 萨德 ” 系统 的 安装 ， 由 着 美方 自由 地 使用 。 如果 这样 ， 中韩关系 将 毫无疑问 地 继续 恶化 ， 建交 25 年来 逐渐 奠定 的 中韩关系 基础 将 销蚀 殆尽 ， 不仅 韩国 经济 必然 遭受 重创 ， 而且 韩国 在 半岛 和 东北亚 的 战略 生

1636

# Search Engine

In [117]:
def naive_search(keywords):
    keywords = keywords.split()
    return [index for index, content in enumerate(data.content) if 
            all(kw in content for kw in keywords)]

In [124]:
%%time
ids = naive_search('清华 大学')
data.content[ids[0]]

Wall time: 18 ms


'图为 速腾 聚创 16 线 激光雷达 在 京东 无人 配送 车上 的 应用 上午 9 点 15 分 ， 京东 无人 配送 车 自主 驾驶 到 中国人民大学 公共 教学楼 ， 为 正在 楼 内 自习 的 李若谷 同学 送 去 了 订单 ， 李 同学 通过 京东 APP 、 手机短信 等 方式 提前 5 分钟 获知 了 快件 即将 送达 的 信息 ， 随后 在 配送 车上 输入 提货 码 ， 配送 车 自动 开启 仓门 — — 这是 京东 618 无人 车 配送 出 的 第一 单 快递 。 无人 配送 车 在 配送 站 完成 商品 装载 后 ， 可以 根据 目的地 进行 自主 路径 规划 ， 寻找 最短 路径 ， 并 规避 拥堵 路段 。 在 行驶 过程 中 遇到 其他 车辆 还 可以 进行 避障 ， 遇到 十字路口 可以 识别 红绿灯 。 当 它 自动 行驶 到 目标 建筑 的 指定 位置 后 ， 会 通过 京东 手机软件 、 手机短信 等 方式 通知 用户 收货 。 用户 可以 通过 人脸识别 或 手机 一键 开箱 功能 便 可取 走 自己 的 包裹 。 早 在 2016 年 的 “ 双 11 ” ， 京东 就 已经 对外 展示 过 无人 配送 车 ， 然而 由于 技术 成本 的 原因 ， 那 时候 无人 配送 车 并 没有 真正 应用 起来 。 「 那 时候 方案 太贵 ， 差不多 可以 买 一辆 奥迪 Q7 （ 60 多万元 ） 。 」 一位 负责 京东 无人 配送 车 调试 的 工程师 表示 ， 经过 半年 多 的 努力 ， 这辆 无人 配送 车 的 成本 已经 基本上 接近 奇瑞 QQ 高配 版 的 价格 （ 5 万元 左右 ） 。 无人 配送 车 造价 降低 了 ， 也 降低 了 配送 的 成本 。 据 了解 ， 以往 人工 配送 一单 的 成本 在 7 元到 10 元 不 等 。 但 现在 即使 将 无人 配送 车 的 电池 损耗 费用 算进去 ， 一单 的 配送 成本 能 控制 在 1.5 元 以内 。 这是 人工 配送 成本 1 / 6 的 价格 ， 不仅 大大 节省 了 人工 配送 包裹 最后 一 公里 的 配送 费用 ， 京东 还 能 通过 这种 技术 进步 将 快递 员 从 繁重 的 体力劳动 中 解放出来 ， 并 进一步 提升 用户 的 购

In [ ]:
# vec，行为文章，列为该文章中每个单词的tfidf
# vec.T，行索引为字典中每个词，行为每篇文章中该词权重，无为0，列为文章列表
vec = vec.toarray()

In [166]:
# 使用转置矩阵，或者是索引列矩阵
all(vec.T[6637] == vec[:,6637])

True

In [173]:
d1, d2, d3 = {1,2,3}, {4, 5, 6, 7, 3}, {1, 3, 4}

In [174]:
reduce(lambda a, b: a & b, [d1,d2,d3])

{3}

In [191]:
def search_engine(query):
    query_vec = vectorizer.transform([query]).toarray()[0]
    try:
        kw = query.split()
        kw_id = [word2index[k] for k in kw]
        docs_id = [set(np.where(vec[:, index])[0]) for index in kw_id]
        # 找出同时包含所有关键字的文章
        merged_docs = reduce(lambda a, b:a & b, docs_id)
        # 根据关键字的tfidf权重与文章对比进行排序，替代方案：jaccard系数等
        sorted_merged = sorted(merged_docs, key=lambda x: cosine(query_vec, vec[x]))
        
        return sorted_merged
    except:
        pass

In [175]:
word2index['清华']

6637

In [206]:
key_word = '美国 军事 司令 指挥'
key_word = '清华 大学'

In [207]:
%time search_engine(key_word)

Wall time: 9 ms


[8651, 8619, 7490, 6242, 7405, 7459, 9068, 6917, 1142, 7544, 5363]

In [208]:
%time naive_search(key_word)

Wall time: 21 ms


[40,
 48,
 381,
 382,
 471,
 582,
 637,
 648,
 887,
 1142,
 1383,
 1622,
 1729,
 1881,
 2300,
 2360,
 2453,
 2485,
 2576,
 3301,
 3416,
 4069,
 4103,
 4450,
 5152,
 5193,
 5231,
 5269,
 5363,
 5468,
 5577,
 5807,
 5865,
 5894,
 5990,
 5999,
 6054,
 6077,
 6150,
 6164,
 6206,
 6242,
 6259,
 6346,
 6367,
 6510,
 6917,
 7013,
 7049,
 7332,
 7349,
 7362,
 7379,
 7397,
 7405,
 7411,
 7427,
 7429,
 7459,
 7490,
 7501,
 7525,
 7544,
 7607,
 7743,
 7938,
 8151,
 8619,
 8651,
 8745,
 8989,
 9053,
 9068,
 9852,
 9949]

### 能用字符串的build-in就不用正则

In [227]:
%%timeit
data.content[8651].replace('清华', '北大')

32.7 µs ± 364 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [225]:
pat = re.compile(r'清华')

In [228]:
%%timeit
pat.sub('北大', data.content[8651])

42.2 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 格式化输出

### Demo

In [284]:
display(Markdown("""
### 标题
> 测试

- 第一个
- 第二个
    - 第一个
    - 第二个
    
**加粗**

*倾斜*

<font color='blue'>这是一段有颜色的输入。</font>

这是什么颜色的呢？

<font color='red'>这是一段红色输入。
"""))


### 标题
> 测试

- 第一个
- 第二个
    - 第一个
    - 第二个
    
**加粗**

*倾斜*

<font color='blue'>这是一段有颜色的输入。</font>

这是什么颜色的呢？

<font color='red'>这是一段红色输入。


In [307]:
def pretty_output(doc, key_words):
    color = ['green', 'red', 'blue', 'yellow', 'pink', 'purple', 'orange']
    kw = key_words.split()
    color_key = np.random.choice(color, len(kw))
    doc = doc.replace(' ', '')
    for i, k in enumerate(kw):
        replacer = f"<font color={color_key[i]}>{k}</font>"
        doc = doc.replace(k, replacer)
    return doc

In [308]:
doc = pretty_output(data.content[8651], '清华 技能 游泳')
display(Markdown(doc))

新华社武汉３月３１日新媒体专电题：醒醒！<font color=orange>清华</font>毕业“硬杠杠”告诉你只顾埋头拼分不行了新华社记者“旱鸭子”将出不了<font color=orange>清华</font>大学的门。对这样的规定你是“点赞”还是“吐槽”？绝大多数人“点赞”，也有一些人提出疑问。无论是“点赞”还是疑问，大家忧虑的都是学生体质下滑的现状，希望综合素质教育早日能真正实现。<font color=orange>清华</font>出新规逼“旱鸭子”下水近日，<font color=orange>清华</font>大学出台新规，从２０１７级本科新生开始，必须通过入学后的<font color=yellow>游泳</font>测试或参加<font color=yellow>游泳</font>课学习并达到要求，否则不能获得毕业证书（特殊情况除外）。在<font color=orange>清华</font>的历史上，也出现过“不会<font color=yellow>游泳</font>不能毕业”的规定。１９１９年，学校曾提出“不会<font color=yellow>游泳</font>不能毕业赴美”的要求。上世纪９０年代末，学校提出<font color=orange>清华</font>学生“至少能游５０米以上”，还专门为二年级学生开设<font color=yellow>游泳</font>课。但将“不会<font color=yellow>游泳</font>不能毕业”写入学生手册作为硬性要求，尚属首次。为什么挑<font color=yellow>游泳</font>作为考查项目？<font color=orange>清华</font>大学体育部负责人刘波解释说：“<font color=yellow>游泳</font>是一项终身运动，有很好的锻炼效果，又是必备的生存<font color=yellow>技能</font>，我们希望借此改变<font color=orange>清华</font>学生体质健康状况下滑的趋势。”“不会<font color=yellow>游泳</font>不能毕业”被认为是努力提升大学生体质、恢复<font color=orange>清华</font>大学体育传统之举，多数受访学生对此项新规表示支持。<font color=orange>清华</font>大学公共管理学院学生王珂说：“我们经常说‘无体育、不<font color=orange>清华</font>’，新规会鼓励更多同学投入体育锻炼，感受运动之美。”也有一些人认为让所有学生都学会<font color=yellow>游泳</font>，有些“不近人情”。对此，<font color=orange>清华</font>大学称，如果学生确实存在因慢性病、皮肤病或心理疾病等原因不适合学习<font color=yellow>游泳</font>的情况，递交医院等相关证明，经学校教务和体育部审核确认后，可用其他体育课程替代<font color=yellow>游泳</font>。这种“硬杠杠”非<font color=orange>清华</font>专属类似的“硬杠杠”并非<font color=orange>清华</font>大学专属。２０１１年开始，厦门大学将<font color=yellow>游泳</font>作为本科生必修课，占１个学分，男生连续游１００米或女生连续游５０米为考试通过，才能获学分。２０１３年，中国人民大学体育部配合学校实施新的人才培养路线图，将“一拳一泳”——太极拳和<font color=yellow>游泳</font>作为核心必修课程。上海大学也规定本科生必须５０米<font color=yellow>游泳</font>达标才能毕业。华南理工大学和中山大学也将<font color=yellow>游泳</font>与毕业绑定，２５米<font color=yellow>游泳</font>达标是两所高校本科生顺利毕业的必要条件。不仅<font color=yellow>游泳</font>达标才能毕业，去年，北京建筑大学出台规定，大一、大二学生一学期长跑锻炼打卡次数不够４５次，体育课成绩将被记为不及格。如果找人代刷，直接被记为不合格。而体育课不及格直接关系学生能否毕业。对于高校这些“硬杠杠”，学生、家长大多赞成，但也有人“喜忧参半”。中山大学法学院学生何思萌说，<font color=orange>清华</font>不是首个将<font color=yellow>游泳</font>与毕业挂钩的高校，中大也有类似规定。“这是一门早就该出台的必修课。不学会<font color=yellow>游泳</font>不能毕业对<font color=orange>清华</font>的学霸们来说影响力还是很大的，实际上大多数学生不是学不会而是懒得学。”上海家长胡女士的儿子正在读高中，<font color=orange>清华</font>是她和孩子计划报考的名校之一。她告诉记者：“看到这个消息有种心里一紧的感觉，这种要求对于怕水的孩子而言，是否公平？会不会阻挡一些想报考<font color=orange>清华</font>学生的脚步呢？”“弱不禁风的幼苗”我们不要记者在采访中了解到，多数高校采取强硬手段“逼”学生锻炼，本意是想改变目前大学生体质健康状况持续下滑的趋势，培养学生养成锻炼的习惯。近年来，学生体质下滑被社会关注甚多。很多体育老师和家长都表示，如今孩子的体质是一届不如一届，近视眼越来越低龄化、肥胖儿童越来越多。武汉一所小学的体育老师告诉记者，就体育课本身而言，如今课时增加了。上个世纪八九十年代，一周只有２节体育课，现在一二年级每周有４节体育课，三至六年级每周有３节体育课，每节课４０分钟。虽说体育课有了保证，但有一种现象却令体育老师们甚感担忧。课间大部分学生都坐在教室里静休。体育课上，运动量大一点，有些孩子就吃不消，跑个４００米就腿发软，爬个六楼就气直喘。武汉一所小学体育老师说：“缺乏锻炼是个普遍问题。”现在的孩子走出校园还要上培优班，连周末都很难到户外好好玩一下，室内活动时间与室外活动时间严重失衡，玩的时间被学习时间大大挤占，仅仅靠每周几节体育课很难提升孩子的体能体质。华中师范大学教授范先佐说，<font color=orange>清华</font>大学不会<font color=yellow>游泳</font>不能毕业之所以引起热议，其实也反映了长期以来中国高校把体育课教学普遍边缘化的现象。他说，不管是<font color=yellow>游泳</font>，还是其他体育锻炼项目列入对学生的考核，最终目的是重视学生生存能力的培养和终身发展。是什么让生活质量越来越高的孩子们，反而越来越不能动了？满腹经纶却又弱不禁风的幼苗，怎能撑得起国家的明天？“如果体育成绩能纳入高考评分体系，学生的体质肯定会随之提升。”采访中，不少体育老师提出这一想法，尽管语气有些无奈。上海纽约大学招办主任周鸿认为，高校这些毕业的“硬杠杠”，特别是<font color=orange>清华</font>大学新规有一个风向标意义，能鼓励学生不要只埋头刷题、拼高分，应该多运动发展自己的兴趣爱好。他说，增强体能体质，对于一个人的生活质量、职业生涯、未来人生发展都会产生比较大的积极影响。（执笔记者：廖君，参与采写记者仇逸、郑天虹、张漫子）

In [311]:
key_word = '体育 足球 新华社 中国 世界杯'
for index in search_engine(key_word):
    display(Markdown(pretty_output(data.content[index], key_word)))
    display(Markdown('---'))

<font color=green>新华社</font>北京３月３１日电新闻分析：２０２６<font color=pink>世界杯</font>出线机会增加<font color=yellow>中国</font><font color=blue>足球</font>如何把握机遇？<font color=green>新华社</font>记者<font color=yellow>中国</font><font color=blue>足球</font>改革推进两年来，足改方案中的“五十条”得到全面贯彻落实，９４％的任务取得阶段性成果，稳步迈出“三步走”战略的第一步；与此同时，世界足坛外部环境正发生变化，２０２６年<font color=pink>世界杯</font>将“扩军”为４８队，亚洲参赛名额几乎翻番，这对于将在２０２１—２０３０年里“奋力实现跻身世界强队”中期目标的<font color=yellow>中国</font><font color=blue>足球</font>而言，既是机遇，也是挑战。为此，<font color=green>新华社</font>记者走访了万达集团、阿里巴巴等国际足联合作伙伴与赞助商，以及<font color=yellow>中国</font>足协、地方足协、俱乐部和知名学者等业内人士。大家一起出谋划策，对推进<font color=yellow>中国</font><font color=blue>足球</font>的改革发展，更有针对性地对内发展青训、对外增加国际足联话语权以及利用好“<font color=pink>世界杯</font>新政”，提出了意见。（小标题）“扩军”带给<font color=yellow>中国</font><font color=blue>足球</font>机遇根据国际足联在３０日公布的“扩军”名额分配的建议方案，亚洲的参赛席位由４．５个增加为８个，接近翻一倍。作为“<font color=yellow>中国</font>最老的<font color=blue>足球</font>人之一”，万达集团董事长王健林认为，“扩军”对亚洲是好事，而<font color=yellow>中国</font>队参加<font color=pink>世界杯</font>的机会应该大增。他说：“扩军是一个机会，至少增加了参与的机会。<font color=yellow>中国</font><font color=blue>足球</font>现在如果认真抓，有机会。”江苏省<font color=yellow>体育</font>局副局长、<font color=yellow>中国</font>足协执委刘彤认为，从<font color=yellow>中国</font><font color=blue>足球</font>发展时机而言，这是个非常好的消息。他表示，<font color=pink>世界杯</font>“扩军”的最大意义是能够让<font color=yellow>中国</font><font color=blue>足球</font>更容易地去接触、学习世界先进水平。“‘扩军’就是缩短进程，这是最大的价值。至于后期带来的机遇、培养，这都是本身要做的，都是要踏踏实实去一步步推进的。”河北华夏幸福<font color=blue>足球</font>俱乐部总经理付强认为，<font color=pink>世界杯</font>“扩军”对<font color=yellow>中国</font><font color=blue>足球</font>会有积极影响。他说，作为一个俱乐部，青训、培养人才、为国家队“输血”是可以努力做的，这是<font color=yellow>中国</font>每个做职业俱乐部人的共同心愿，希望通过自身努力能在未来帮助<font color=yellow>中国</font><font color=blue>足球</font>实现发展。“或早或晚，我们能够触到<font color=pink>世界杯</font>。”除了出线机会增加外，扩大话语权也是<font color=yellow>中国</font><font color=blue>足球</font>需要把握的机遇。阿里巴巴是国际足联世俱杯从２０１５到２０２２年的独家冠名赞助商。阿里<font color=yellow>体育</font>副总裁李峰介绍，从与国际足联的交流中感受到，国际足联强烈希望<font color=yellow>中国</font><font color=blue>足球</font>能够更加融入国际足坛事务中。李峰说：“他们（国际足联）觉得<font color=yellow>中国</font><font color=blue>足球</font>不可或缺，因为<font color=yellow>中国</font>是个大市场。但反过来，他们给我们的感觉，还是觉得<font color=yellow>中国</font><font color=blue>足球</font>不懂得运用<font color=blue>足球</font>规律和市场规律去获取价值。”首都<font color=yellow>体育</font>学院校长钟秉枢认为，国际足联既然决定“扩军”，就是逐步在把“发展杠杆”向范围更广的<font color=blue>足球</font>发展<font color=yellow>中国</font>家倾斜。作为其中重要的代表，<font color=yellow>中国</font><font color=blue>足球</font>应该主动公关，增加话语权。（小标题）扎实推进足改三方面把握机遇<font color=yellow>中国</font>足协副主席李毓毅曾表示，在足改两年稳步迈出了“三步走”战略的第一步后，下一步将为把工作重心从第一阶段转向第二阶段做好充足准备。而第二阶段的任务也将是要求取得成绩的四大硬指标——“青少年<font color=blue>足球</font>人口大幅增加，职业联赛组织和竞赛水平达到亚洲一流，国家男足跻身亚洲前列，女足重返世界一流强队行列”。这个阶段在《<font color=yellow>中国</font><font color=blue>足球</font>中长期发展规划（２０１６－２０５０年）》中设定的时间段为２０２１－２０３０年，正是涵盖２０２６<font color=pink>世界杯</font>的“扩军年”。如何顺利完成中期目标？业内人士给出三条建议。建议一：用三大“硬杠杆”去考核落实万达集团董事长王健林认为，足改两年来取得的成绩有目共睹，但青训工作依然任重道远。对于如何把握规律地提升<font color=yellow>中国</font><font color=blue>足球</font>基础，王健林建议，用三个“硬杠杆”来考核约束——<font color=yellow>体育</font>设施、<font color=blue>足球</font>人口、青少年赛事的体系和举办。他说：“２０２６年的机会，还有７－８年，现在抓来得及，就是抓１４－１５岁的年龄段，那个时候就是出成绩的时候，留给我们的时间是够的，但必须从现在紧抓，大量发现苗子。这三个作为‘硬杠杆’来考核约束。如果现在开始真抓实干，２０２６年一定有机会。”对于场地设施，王健林建议，<font color=yellow>中国</font>足协要指导各地足协，真正把精力投入到青少年培养上。其中，每个地方要有多少个<font color=blue>足球</font>学校、<font color=blue>足球</font>场地，特别要作为硬指标来约束各级协会。对于<font color=blue>足球</font>人口，王健林说，<font color=yellow>中国</font>青少年<font color=blue>足球</font>人口只要过了百万，<font color=yellow>中国</font><font color=blue>足球</font>水平就有机会跻身亚洲一流行列。此外，他建议，从１２岁年龄段起步的全国性青少年比赛，也需要成体系地设计和举行，因为好苗子的发现、水平的提升，都要靠赛事的磨炼和拉动。建议二：青训体系要统一理念最根本还要靠自己首都<font color=yellow>体育</font>学院校长钟秉枢指出，青训两年来取得积极进展，但也存在着“多点开花”、没有统一理念的问题。随着多元化的机构出现，<font color=yellow>中国</font><font color=blue>足球</font>的青训市场没有统一标准，这边学的是“巴萨理念”，那边请的是“皇马风格”的教练。他说：“我们当然要引进青训体系，但每个体系的指导思想是不一样的。荷兰的、德国的，每个国家的青训体系是根据自身的特点构造起来的，引进<font color=yellow>中国</font>后，谁来做<font color=yellow>中国</font>化，有没有统一的<font color=yellow>中国</font>化布局？这也是为什么这么多青年队出国训练，回来后没有建立<font color=yellow>中国</font><font color=blue>足球</font>自己的体系的原因。”钟秉枢建议，<font color=yellow>中国</font>足协要引导“技术走向”，出台相应的大纲和规范文件。他举例，<font color=blue>足球</font>强国为什么做得好？他们各个单项组织下面都设有一个技术总监，负责这个项目从上到下的技术问题，国家队的选拔、青训的标准、技术的规范等。<font color=yellow>中国</font><font color=blue>足球</font>的青训工程也需要这样的体系。谈到在校园<font color=blue>足球</font>领域教育部和<font color=yellow>中国</font>足协的合作，钟秉枢建议，<font color=yellow>中国</font>足协在技术层面应加强指导，行业协会应统一管理。而且，社会上出现的多元化<font color=blue>足球</font>培训机构，必须在<font color=yellow>中国</font>足协的考核和管理下做好整合，才有可能在不同年龄组、不同渠道上，为提高国家<font color=blue>足球</font>水平做出统一贡献。王健林也建议，<font color=yellow>中国</font>的青训从根本上还是要靠自己。“最根本的还是国内青训和人口要跟上。没有一个大国的<font color=blue>足球</font>运动是靠别国来培养的。”建议三：<font color=blue>足球</font>外交或可利用企业资源阿里<font color=yellow>体育</font>副总裁李峰认为，作为国际足联合作伙伴，企业有很大的资源，可以为<font color=yellow>中国</font><font color=blue>足球</font>所用。如果<font color=yellow>中国</font><font color=blue>足球</font>明确交给企业一个任务，企业一定会认真去考虑。王健林认为，如果足协愿意和企业合作，万达以及盈方与国际足联的合作关系可以为<font color=yellow>中国</font><font color=blue>足球</font>发展所用。“我们和国际足联也在磋商，除了<font color=yellow>中国</font>杯，还要给我们未来更大赛事去做。当然如果能为<font color=yellow>中国</font><font color=blue>足球</font>做事，我们何乐而不为？肯定愿意做。”对于如何让“企业＋足协”强强联合、扩大话语权，钟秉枢提出两点建议——输送打破界限的人才进入足联、拉近与有投票权协会的关系。他说，“脱钩”后的<font color=yellow>中国</font>足协，作为一个社团组织，更应大规模输送人员到国际足联、亚足联。以前只能从总局和（足管）中心物色人选，现在应广泛寻求有能力的社会各界人士，通过各种机会输送，提高话语权。另外一个角度，即便人没有进入国际组织，也需要联系、沟通其他协会有投票权的委员、执委，这个是需要大量做工作的。（完）（采写记者：岳东兴、蔡拥军、树文、韦骅、王恒志、王浩宇、杨帆、张逸飞）

---

国内新闻（科技）中科院研究人员发现“会吃塑料的真菌”（科技）在南海洋陆过渡带的基底岩石中寻找“时间胶囊”（经济）广东：２０１７年完成１００个智能改造项目（关注春耕备耕）“湘中粮仓”：别样春耕勾勒农事新图景（法治）珠海通报“当街宰杀海豚”事件初步调查结果：渔民拖回已死亡“白忌鱼”售卖国内重要稿件预告（3月31日下午）（环境）王桂平：“观鸟”也致富（医卫）陕西将执行新版医疗服务价格方案以药养医成为历史（环境）北京今年将启动多条绿道建设建成后总长度达近500公里（财经·行情）人民币对美元汇率中间价下跌１０４个基点（财经·动态）海南取消重大疾病及支付学费提取住房公积金等规定（科技）专访ＤＮＡ之父詹姆斯·沃森：人类离治愈癌症还有多远？（经济）在华留学生可深度对接中关村高新技术企业（光盘行动在身边）“再也不用拿小孩的奶粉钱随礼了”——湖北房县不让人情债成为脱贫路上的绊脚石（社会·调查）醒醒！清华毕业“硬杠杠”告诉你只顾埋头拼分不行了（新华简讯）<font color=red>中国</font>航天科工集团计划发射156颗小卫星将基本实现覆盖全球的宽带互联网接入（新华简讯）3月<font color=red>中国</font>制造业PMI升至51.8%（服务·生活）西藏经历新世纪第１６个“暖冬”（经济）北京大幅下调有形建筑市场交易服务费平均降幅达46％（经济）北京将取消、停征、减免多项行政事业性收费年减负约26.6亿元（走基层听民声）搭着无人机，“老农民”飞向好日子（社会）河南启动困难群众大病补充医疗保险工作７３４万贫困人口受益国内重要稿件预告（3月31日上午）（社会）湖北麻城脚手架垮塌事故致9死6伤事故调查组已进驻（经济）新闻分析：从<font color=red>中国</font>服装到<font color=red>中国</font>时尚，还要走多久？（社会）湘江干流迎来今年首个洪峰湖南局地出现山体滑坡（环境）清明假期京津冀及周边地区将遭遇一轮空气污染过程（科技）<font color=red>中国</font>航天科工集团计划发射156颗小卫星将基本实现覆盖全球的宽带互联网接入（服务·健康）甘肃通报17批次不合格药品（社会·广角）河南省南阳市副市长和学民涉嫌严重违纪接受审查（社会·广角）厦门“飓风肃毒”行动缴获各类毒品１０６公斤（时政）全国政协机关开展义务植树活动（时政）国务院印发《<font color=red>中国</font>（辽宁、浙江、河南、湖北、重庆、四川、陕西）自由贸易试验区总体方案》（时政）国务院印发《全面深化<font color=red>中国</font>（上海）自由贸易试验区改革开放方案》（外事）外交部就习近平主席对芬兰进行国事访问并赴美国举行中美元首会晤举行中外媒体吹风会通稿线路3月31日发稿目录（一）对外新闻（港澳台）台湾“2·13”游览车事故33名罹难者已完成理赔2名官员遭记大过处分（财经·行情）人民币市场汇价（３月３１日)国际新闻国际重要稿件预告（经济）文莱经济２０１６年萎缩２．５％（财经）汽车安全技术助德国汽车零部件巨头收入大涨（经济）捷克央行宣布继续执行外汇干预政策（科技）综述：各国专家热议北极科学发展（科技）日本大幅解禁福岛避难区但居民返乡意愿低（科技）俄方专家热切希望与中方合作开发北极（文化）特写：在波兰幼儿园感受“<font color=red>中国</font>戏剧文化月”（文化）摩纳哥今夏将举办《紫禁城》主题展览<font color=red>新华社</font>快讯：据外电报道，巴基斯坦巴勒吉纳尔市３１日发生一起爆炸，造成至少５人死亡，另有４０人受伤。（国际）美官员说美不再优先考虑叙总统巴沙尔去留问题（国际）美国国务卿蒂勒森访问土耳其强调合作打击恐怖主义（国际）葡萄牙和智利将加强双边经贸合作（国际）全国人大西藏代表团访问瑞士（国际观察）李克强澳新之行扩大“一带一路”朋友圈（科技）首届中比科技创新对话在比利时举行（科技）详讯：“二手”火箭成功发射回收美公司欢呼航天“革命”（国际）俄白总理会谈未能解决双方油气争端（国际）以色列２０多年来首次批准在约旦河西岸建新定居点（国际）联合国官员承诺帮助塔吉克斯坦应对可能的难民涌入问题（服务）国航推出马德里—北京直航新机型和新航线（科技）力主合作开发的国际北极论坛在俄闭幕国际重要稿件预告（科技）俄空天部队开始部署新一代空间管制系统（科技）英国大力推动无人驾驶技术发展（文化）泰国今年泼水节将展示多样文化（国际）３名被南苏丹反叛武装劫持的印巴石油工人抵达喀土穆（国际）巴西前众议长涉巴油腐败案被判１５年监禁（国际）白宫说特朗普期待与习近平会晤以规划中美关系前进路线<font color=blue>体育</font>部（<font color=blue>体育</font>）南方地区跨项选拔冬季项目运动员的“后顾之忧”<font color=blue>体育</font>重要稿件预告（<font color=blue>体育</font>）黑龙江省将开展“我要上全运”大众系列项目争霸赛（<font color=blue>体育</font>）上海上港Ｕ１８梯队巴西青训正式开营（<font color=blue>体育</font>·国际<font color=orange>足球</font>）各大洲博弈皆大欢喜附加赛或定最后两席——２０２６<font color=orange>世界杯</font>“扩军”名额解析（<font color=blue>体育</font>）新闻分析：２０２６<font color=orange>世界杯</font>出线机会增加<font color=red>中国</font><font color=orange>足球</font>如何把握机遇？（<font color=blue>体育</font>）澳橄榄球俱乐部宣布与<font color=red>中国</font>南航签署赞助协议（<font color=blue>体育</font>）希望影响一代<font color=red>中国</font>冰球运动员和球迷——访ＮＨＬ总裁贝特曼<font color=blue>体育</font>重要稿件预告<font color=red>新华社</font>技术局

---

“我喜欢野生的力量”——一个“９０后”女孩的戏剧奇缘百件文物中的人类文明对话特稿：春天来了，去看老树画花（新华简讯）马里宣布再次实施紧急状态（新华简讯）韩国法院批准逮捕前总统朴槿惠（新华简讯）普京否认俄罗斯影响美国大选国防部：首艘国产航母舾装工作进展顺利隋文静／韩聪首夺花样滑冰世锦赛双人滑金牌广州青岛调控加码：不动产证满两年方可交易扰乱市场的“大鳄”，要坚决逮住！证监会开出“史上最大罚单”广州青岛调控加码：不动产证满两年方可交易第三批自贸区列近千项创新清单金融探索再获突破文化产业扬帆出海创建<font color=purple>中国</font>的“达·芬奇密码”公募基金市场迎来“小阳春”一季度规模业绩齐收涨智慧模式成转变医疗服务突破口一罚到底３４．７亿元！鲜言为啥领了证监会“史上最大罚单”？（新华财经）广州青岛调控加码：不动产证满两年方可交易（新华财经）公募基金市场迎来“小阳春”一季度规模业绩齐收涨（新华财经）第三批自贸区列近千项创新清单（新华财经）文化产业扬帆出海创建<font color=purple>中国</font>的“达·芬奇密码”（新华财经）智慧模式成转变医疗服务突破口头衔“总统助理”美“第一女儿”成白宫正式员工广州青岛调控加码：不动产证满两年方可交易朝鲜公民死亡事件协议达成双方解除出境禁令一纸逮捕令朴槿惠进看守所你的衣橱跟上“潮流”了吗？２０１７年秋冬流行趋势面面观５年前撞人逃逸泰国红牛继承人再“爽约”传讯３月３１日《参考消息》全球新闻ＴＯＰ５遭“民意”劝退法两位总统候选人坚持不放弃ＣＢＡ总决赛数据酷｜前瞻①总决赛四度相逢谁称王聊聊关于郎平续约你最想知道的五件事韩国法院批准逮捕前总统朴槿惠朝鲜与马来西亚就朝鲜公民死亡事件达成协议东京国际宠物展开幕聚焦宠物让生活更美好瞬间看世界２０１７０３３１现场｜韩民众阻拦“萨德”：我们获得了暂时的胜利美国高校人才“跳来跳去”，制衡靠“软调节”新华时评：“脱欧”不是欧洲的末日解码高校“挖人大战”浙江温州：畲乡风情醉游人壮乡歌节“三月三”美宇航员创女性太空行走次数新纪录“声声”不息的世界之窗上海电影译制厂迎来６０华诞专家访谈：中美关系需要“新战略框架”醒醒！清华毕业“硬杠杠”告诉你只顾埋头拼分不行了云南富源：又到山花烂漫时新闻分析：从<font color=purple>中国</font>服装到<font color=purple>中国</font>时尚，还要走多久？湖北麻城脚手架垮塌事故致９死６伤事故调查组已进驻<font color=purple>中国</font>航天科工集团计划发射１５６颗小卫星将基本实现覆盖全球的宽带互联网接入独家｜全景鸟瞰中美元首即将会晤的海湖庄园李克强澳新之行扩大“一带一路”朋友圈人类太空史首次成功发射回收“二手”火箭美公司欢呼航天“革命”数说共享单车的“成长”与“烦恼”〔<font color=green>新华社</font>快讯〕快讯！许利民担任<font color=purple>中国</font>女篮主教练特朗普期待与习近平会晤以规划中美关系前进路线新闻分析：２０２６<font color=purple>世界杯</font>出线机会增加<font color=purple>中国</font><font color=orange>足球</font>如何把握机遇？从北到南，鸟瞰春耕美官员：解决叙危机美不再优先考虑巴沙尔去留问题（<font color=purple>中国</font>网事·感动２０１７）“奶爸仁医”胥亦龙：只想着抢救病人，忘了还背着孩子飞阅西成客专四川段云南维西县傈僳族山乡的变迁中俄界湖迎“鸟汛”“游泳圈”难甩走？告诉你三大中医减肥妙招ＡＰ１０００通过英国通用设计审查我国核电自主化依托项目稳步推进机构：北京写字楼空置率或持续攀升樱花绽放温哥华韩国“世越”号沉船抵达木浦新港九五丝御·邓兆萍时装发布会在京举行专家：以标准化设计引领住宅产业化发展<font color=purple>中国</font>科研如何走向世界前沿：解码元英进团队国家扶贫开发重点县的教育扶贫实践潮白河上摆渡人每日穿梭京冀间刘盛毅·黄柏青联合手袋秀在京举行转投马克龙法前总理“走棋”引争议第三批自贸区列近千项创新清单２０１７年谷歌超越苹果登顶美国５００强企业排行榜新华微视评｜科学管理“小红车”文明出行你我他拜仁将在深圳建设全日制<font color=orange>足球</font>学校满眼繁花春似锦，出门俱是赏花人商务部：上海自贸试验区为其他自贸试验区建设发展提供改革创新借鉴“飞阅”壮族三月三各族群众舞翩跹“一条逮捕令”朴槿惠被送看守所西湖龙井上市资深茶农教你看门道恐龙也能这么好玩！快来寻宝！浙江侦破２２年前宁波绿洲珠宝行特大杀人抢劫案想长寿？养宠物聚焦人与宠物的“双赢生活”来美国小镇感受<font color=purple>中国</font>民间艺术挖挖挖！“扶贫资金”暗度陈仓？宰相后裔：先祖不留财产留家规科学家成功编辑人类胚胎基因治疗遗传疾病有望成真清明节前带你去看“杏花村”（热点鲜报）“<font color=purple>中国</font>网事·感动２０１７”网络感动人物评选一季度启动侵华日军南京大屠杀遇难同胞纪念馆又添新证范长龙、常万全分别会见瑞士国防部长外汇局的国际收支年报都说了啥（<font color=purple>中国</font>网事）哪能关了“电炉子”点起“煤炉子”？北京房山区破解老百姓“煤改电”困局饮食文化碰到法律红线要“忌口”为了心间那片“挪威森林”宁夏回族自治区人民政府原副主席白雪山受贿案一审开庭巴基斯坦市场遭袭２２死７０伤（热点鲜报）西藏查获一重大濒危动物制品案件缴获２０张雪豹皮别让器官捐献卡在“后顾之忧”上（<font color=purple>中国</font>网事）５Ｇ发展进入快车道，新技术将如何改变生活？安徽霍邱水门塘千亩水面万顷波（<font color=purple>中国</font>网事）南京“侠女护士”怒斥“医托”获赞，若再遇“医托”她还会“出手”（新华简讯）美国宇航员打破女性太空行走次数纪录（新华简讯）力主合作开发的国际北极论坛在俄闭幕“为爱发声”专项基金在北京发布探索农业供给侧改革的那些故事２０１７国际田联钻石联赛上海站公布首批明星阵容（新华简讯）巴基斯坦一市场遭袭致２２死７０伤（新华简讯）美官员说美不再优先考虑叙总统巴沙尔去留问题要充分尊重<font color=orange>足球</font>市场规律——专访体奥动力董事长李义东贝加莫大学里的一堂<font color=purple>中国</font>文学影视课独家：俯瞰中美元首即将会晤的海湖庄园“决心”号上的南海黄昏代建模式将助力房地产开发迈入“品质服务”时代李克强主持召开国务院第七次全体会议决定任命林郑月娥为香港特别行政区第五任行政长官〔<font color=green>新华社</font>快讯〕夜览｜天津限购新政：非本地户籍购房需连续缴两年社保资深<font color=red>体育</font>媒体人为晋江打造<font color=red>体育</font>城市“支招”三问美国在巴沙尔去留问题上态度变化（热点鲜报）几代“好猎手”追击“老狐狸”“浙江第一悬案”背后竟有这样的故事哪能关了“电炉子”点起“煤炉子”？北京房山区破解老百姓“煤改电”困局日本植树访华团持续３２年悼念南京大屠杀遇难者我国自贸试验区形成“１＋３＋７”格局西藏查获一重大濒危动物制品案件缴获２０张雪豹皮自贸试验区新增７个成员，释放什么信号什么样？啥特点？怎么干？——<font color=green>新华社</font>记者探访第三批自贸试验区新闻链接：７个新设自贸试验区大盘点短评：自贸试验区扩围彰显开放新姿态“<font color=purple>中国</font>网事·感动２０１７”网络感动人物评选一季度启动辽宁省两名厅官被开除党籍歌星比伯巴西涂鸦遭传讯生命的馈赠——我国人体器官捐献登记数量激增背后的喜与忧什么样？啥特点？怎么干？——<font color=green>新华社</font>记者探访第三批自贸试验区（热电鲜报）３名在珠海涉宰杀海豚者被警方传唤“奶爸仁医”胥亦龙：只想着抢救病人，忘了还背着孩子中铁十一局：精心打造汉十高铁精品梁皖南深山“淘宝村”里的“淘宝”小店缅怀高尚的生命选择<font color=green>新华社</font>技术局

---

In [325]:
%%time
key_word = '美军 战争 指挥'
for index,id_ in enumerate(naive_search(key_word), start=1):
    display(Markdown(f'**Search Result {index} => Document Index : {id_}**'))
    display(Markdown(pretty_output(data.content[id_], key_word)))

**Search Result 1 => Document Index : 820**

参考消息网6月12日报道日媒称，5月30日，韩国总统文在寅指示要彻查四台“萨德”发射架秘密运抵韩国的事实。据《日本经济新闻》网站6月12日文章称，“萨德”反导系统被用于应对朝鲜核导弹攻击的王牌，由驻韩<font color=pink>美军</font>操作使用。在朴槿惠执政时期，美韩达成了引入“萨德”系统的协议。文在寅对“萨德”一直持谨慎态度，在刚刚上台之际发生了在总统不知情的情况下“萨德”发射架被秘密运进韩国的事件。在文在寅要求彻查指示的背后，隐藏着他与驻韩<font color=pink>美军</font>长达11年的因缘。摆脱美国实现“自主国防”6月5日，韩国总统府宣布了调查结果。文在寅之所以没有看到发射架运抵韩国的相关报告，是因为国防部国防政策室长（中将）指示下属从报告中删除了相关内容，试图进行隐瞒。行政透明正是文在寅批评朴槿惠前政府实行“密室政治”的主要武器。但在记者会上公开政府内部的瞒报行为和调查命令极为罕见。在韩国，对于文在寅这一做法的理由出现了各种猜测，有人认为“由于新政府的人事安排频现违规行为，此举意在转移视线”、还有人认为这是“为军队人事和组织的深度改革铺平道路”。如果将视线延伸至总统任期的5年之后，将看到不同的风景。“我们的未来愿景应该是打造海洋强国”，在发出调查指示的次日——5月31日，文在寅在出席国内活动时如此表示，表明了将大幅增加投资提升海军战斗力。韩国要争取实现的“自主国防”，其核心就是被写入文在寅竞选纲领的、早日拿回战时作战<font color=blue>指挥</font>权。在朝鲜<font color=pink>战争</font>之后，“有事”之际的韩国军队<font color=blue>指挥</font>权一直由<font color=pink>美军</font>掌握。如果美韩认为“有可能出现重大紧张状态和军事干预”的情况下，韩国军队的约63万名士兵将接受驻韩<font color=pink>美军</font>司令的<font color=blue>指挥</font>。文章称，而提出自主国防路线的前总统卢武铉曾试图让<font color=blue>指挥</font>权重归韩国军队。在卢武铉任内的2007年，韩国与美国就2012年4月将<font color=blue>指挥</font>权移交给韩国军队达成协议。但韩国的能力受到怀疑，在李明博上台后，<font color=blue>指挥</font>权的回归被推迟至2015年12月，到了朴槿惠执政时期，决定在事实上无限期推迟。

**Search Result 2 => Document Index : 1636**

近期，来自韩美有关“萨德”问题的消息很多，不断冲击着中国的舆论和中国人民的心理底线。美国总统特朗普6月8日与国务卿蒂勒森、国防部长马蒂斯等核心高官及参谋一起就文在寅政府决定先环评再部署“萨德”进行了讨论，对此，青瓦台高官次日就“萨德”争议表示，韩国政府无意从根本上改变韩美同盟之间约定的事项。韩国新任防长提名人宋永武所谓的买下“萨德”的议案重新抬头，等等。围绕“萨德”问题的争议似乎变得越来越复杂，形势越来越严峻。文在寅政府在“萨德”问题上的三种选择文在寅政府今后如何处理“萨德”问题，将被国际社会所关注，对他来说或许是一个痛苦的抉择。不管问题有多么复杂，一般来讲，不外乎有以下三种可能性：一是完全按照中方提出的要求，撤除“萨德”系统。客观上，在朝鲜半岛乃至东北亚的紧张局势下，韩国存在一定的支持部署“萨德”的民意，特别是来自现有韩美同盟机制的压力。朝鲜<font color=red>战争</font>结束以来，韩国军队的战时<font color=orange>指挥</font>权至今尚在<font color=red>美军</font>手中。韩国前总统卢武铉当政期间，已经与美方达成协议将战时<font color=orange>指挥</font>权适时交还韩方，后来，保守的李明博政府和朴槿惠政府将此期限相继延期。所以，在朝韩互信远未建立、同盟机制压力与日俱增的背景下，文在寅似乎很难做到撤除“萨德”。冒着与美方翻脸的风险，文在寅恐怕不敢。二是完全满足美方的要求，继续完善“萨德”系统的安装，由着美方自由地使用。如果这样，中韩关系将毫无疑问地继续恶化，建交25年来逐渐奠定的中韩关系基础将销蚀殆尽，不仅韩国经济必然遭受重创，而且韩国在半岛和东北亚的战略生存空间会受到严重的挤压。三是以拖待变，试图寻找最有利于韩方的机会。“萨德”是战略武器，不仅会直接威胁中国与俄罗斯的战略安全，还会打破东北亚地区的战略平衡，在理论上来说，文在寅政府很难找到使中俄与美韩都能接受的某种平衡方案。所以，文在寅只能等待和找到最有利于韩方的机会。显然，这种选择的可能性最大。文在寅政府或在“萨德”问题上玩“太极”战略根据韩国《环境影响评估法》及施行细则，环境影响评估分为小规模环境影响评估、战略环境影响评估和正式环境影响评估，一旦成为正式环境影响评估对象，可能将需要两年左右时间。青瓦台6月8日发布消息说，“萨德”部署用地属于“正式环境影响评估”对象。韩总理办公室8日当天成立了跨部门联合工作组，将对疑似逃避环境评估等过程中出现的问题进行追加调查，同时商讨实施适宜的环境评估等。韩政府启动环评程序，延缓了“萨德”系统的实战部署，实现了以拖待变的目的。但是，未必对中国肯定有利，如果环评未通过，最终撤除“萨德”，当然是中国希望看到的结果。如果环评通过，那么部署“萨德”就有了法律上的依据，将来更难拆除，反而对中国更为不利。所以，环评是一把双刃剑，文在寅政府可能是在玩“太极”战略，中国应该严防在“萨德”环评问题上被忽悠。韩国总统文在寅6月11日提名5名部长人选，其中国防部长提名人宋永武引发关注。韩国《中央日报》11日称，宋永武在大选前曾公开表示，韩国军方可以考虑拥有“萨德”并自主运行，这意味着韩军可从美购买“萨德”，“中国反对萨德的理由就会消失”。显然这是一种错误想法，因为不管哪方运行，在韩部署“萨德”就势必对中国国家安全构成实质性威胁。况且，韩军购买“萨德”自主运行的方案是很不现实的，因为一是“萨德”技术不是韩国自己开发的，在技术上还必须依靠美国;二是韩国还没有从美国手里收回战时<font color=orange>指挥</font>权，也就是说，韩国无权<font color=orange>指挥</font>实施“萨德”的实战操控;三是美国不可能将“萨德”尖端技术转卖给韩国。因此，这一方案也是一种不切实际的忽悠。中国应对韩国持续施加足够大压力并给以战略引导自文在寅就任总统以来，韩国在“萨德”问题上的政策走向就成为中国舆论关注的焦点。由于朴槿惠政府不顾中方的反对执意部署“萨德”，使中韩关系跌至1992年建交以来的冰点。因此，不少中国人对文在寅政府充满期待，并希望能够推翻朴槿惠政府引“萨德”入韩的决策，这种心情是可以理解的。但是，现实是不容乐观的，任何一个国家战略抉择的改变需要外部推动力，或者说外部压力。对文在寅政府来说，无论是部署或撤除“萨德”都会面临来自韩国国内和国外的反对与压力，这就要看哪方的压力更大，文在寅应该选择牺牲压力比较小一方的利益。因此，中国应联合俄罗斯制定周密的反制战略来应对“萨德”，必须综合使用政治、经济、军事和外交等实力，施加足够大的压力，才有可能推动文在寅政府改变战略方向，才会出现解决“萨德”问题的可能性。现阶段或短期内最现实的目标是迫使文在寅政府调整政策，既不公开撤消也不实战部署“萨德”系统，将“萨德”冷冻起来，使其成为“植物人”，待时期成熟之时，再正式撤除，同时警告韩国在战略上切不可忽悠中国。“萨德”问题正考验着有关各方的实力和耐力，是一场战略和意志的较量。(作者为本栏目特约评论员、浙江大学韩国研究所客座研究员、中国社科院朝鲜半岛问题专家)

**Search Result 3 => Document Index : 2833**

点击图片进入下一页贾法勒基地一隅。《解放军报》资料图片美海军设在巴林的贾法勒基地，位于巴林最大岛屿巴林岛，距巴林首都麦纳麦约8公里。巴林是海湾地区最小的国家，由36个大小不等的岛屿组成，但其战略位置极其重要。早在1935年，英国就在巴林建立了贾法勒海军基地并驻军，以保持对巴林及周边地区的控制及威慑。1971年，巴林独立成为主权国家。此后，英国放弃了贾法勒海军基地，而美国则通过与巴林政府签署协议，接管了该基地。虽然巴林战略位置极其重要，但直到海湾<font color=blue>战争</font>前，<font color=orange>美军</font>驻扎在贾法勒海军基地的人员和装备都很少。海湾<font color=blue>战争</font>时，<font color=orange>美军</font>在贾法勒海军基地进行了大规模的海军舰艇集结，贾法勒基地在冷战后的战略位置得以凸显。“9·11”事件后，该基地对<font color=orange>美军</font>进行阿富汗<font color=blue>战争</font>和伊拉克<font color=blue>战争</font>发挥了重要作用。随着近几年极端组织“伊斯兰国”肆虐，该基地又成为<font color=orange>美军</font>在伊拉克和叙利亚打击恐怖主义的前沿阵地。如今的贾法勒基地驻有美海军第五舰队司令部、<font color=orange>美军</font>中央总部海军司令部、中央总部陆战队分遣队、特种作战<font color=green>指挥</font>与控制分遣队等部门，统辖在波斯湾、红海、阿曼湾、亚丁湾和阿拉伯海等地活动的海军部队，是美海军在中东地区的大本营。<font color=orange>美军</font>以设在巴林岛的贾法勒基地为大本营，可控制整个波斯湾及霍尔木兹海峡出入口，也可扼守曼德海峡、苏伊士湾和亚喀巴湾。可以说，一旦波斯湾和霍尔木兹海峡“有事”，贾法勒基地就是驻中东美国海上力量的作战<font color=green>指挥</font>中心。

**Search Result 4 => Document Index : 5113**

参考消息网6月21日报道英国《独立报》网站6月12日发表题为《美陆军上尉称<font color=green>美军</font>“不再是世界最好的军队了”》的报道称，一名美国陆军上尉称，<font color=green>美军</font>或许不是世界上最好的军队，因为“它的战斗准备不如多国伙伴”。J·斯科特·梅茨上尉在一家陆军专业期刊上撰文说，他感觉其他国家的军队“在战术方面，目前在连级及以下级别比我们强，因为我们在常驻基地训练不够”。他的结论是盟军和“一些潜在的敌人”比<font color=green>美军</font>准备得更充分。梅茨将此事归咎于“战备”——一个涵盖从确保士兵接受最新疫苗接种、武器认证到国防部老旧的计算机系统中保存有合适的文书等各种事务的术语。他在文中举的一个例子是，<font color=green>美军</font>一个野战排在每次交战中都不知道如何利用地形优势，在一次军演中被外国同行智胜。他认为，这是在抵达多国联合战备中心（JMRC）之前就应该在常驻基地训练的内容。梅茨说，军队高级领导人把战备作为最优先考虑事项，但基层单位和<font color=blue>指挥</font>官要负担过重的训练要求，没有足够的时间为行政工作做好充分准备。自上世纪90年代以来就报道<font color=green>美军</font>新闻的《外交政策》双月刊专栏作家汤姆·里克斯撰文说，梅茨上尉“在向当权者说真话”。直到最近，梅茨还驻扎在德国霍恩费尔斯的多国联合战备中心，这令他对<font color=green>美军</font>有独特的看法——但有些局限性，因为他军衔不高，而且也只是在记述他在陆军中而非整个军队中的视角。这篇文章——十有八九通过了领导这位现役上尉的各级<font color=blue>指挥</font>官的审查——被军界许多人称赞直言不讳，尤其称赞他对美国外交政策的现状、目前在阿富汗的<font color=pink>战争</font>和在伊拉克、叙利亚和其他一些地方的战事的看法。里克斯撰文说，梅茨上尉的文章是一篇“挑战主流观点的尖锐和信息丰富的文章”。发表这样的文章是军事专业刊物职责的重要部分，但它们并不总是履行这一职责。（编译/王海昉）点击图片进入下一页2017年1月10日，美国海军陆战队在空地协同战斗中心举行代号“协作训练2017”的演习。

Wall time: 33 ms


In [333]:
sys.getsizeof(vec) / 1024 / 1024

762.9395599365234

In [347]:
sys.getsizeof(data.summary) / 1024 / 1024

1.682952880859375

In [336]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 3 columns):
content    10000 non-null object
title      10000 non-null object
summary    4440 non-null object
dtypes: object(3)
memory usage: 632.5+ KB


In [339]:
%%memusage data

UsageError: Cell magic `%%memusage` not found.


In [344]:
vec.shape

(10000, 10000)